In [1]:
import time
import os
import pickle

from simtk.openmm import XmlSerializer
from simtk import openmm
from simtk.openmm import app, unit
import openmmtools

# Check computation time for setUseDispersionCorrection(True) for only the NonbondedForce that contains nonalchemical atoms, but not the CustomNonbondedForce that contains alchemical atoms

Use long-range env

In [9]:
with open('rbd_ace2_example/rbd_ace2_hybrid_system.xml', 'r') as f:
    system = XmlSerializer.deserialize(f.read())
with open('rbd_ace2_example/rbd_ace2_state.xml', 'r') as f:
    state = XmlSerializer.deserialize(f.read())

In [10]:
system.getForces()

[<openmm.openmm.MonteCarloBarostat; proxy of <Swig Object of type 'OpenMM::MonteCarloBarostat *' at 0x2b04dbf2eed0> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b04dbf2e1b0> >,
 <openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b04dbf2e0f0> >,
 <openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2b051d42dba0> >,
 <openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b051d42de10> >,
 <openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2b051d42d8d0> >,
 <openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b051d42dcf0> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b051d42d120> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of t

In [11]:
system.getForce(7).getUseDispersionCorrection()

True

In [12]:
system.getForce(8).setUseLongRangeCorrection(False)

In [13]:
# Set up integrator
integrator = openmmtools.integrators.LangevinIntegrator(temperature=300 * unit.kelvin, 
                                                        collision_rate=1 / unit.picoseconds, 
                                                        timestep=4 * unit.femtoseconds)


In [14]:
# Set up context
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*state.getPeriodicBoxVectors())
context.setPositions(state.getPositions())
context.setVelocitiesToTemperature(300 * unit.kelvin)

In [15]:
nsteps = 100
initial_time = time.time()
for step in range(nsteps):
    context.setParameter('lambda_sterics_insert', (step + 1) * 0.01)
    integrator.step(1)
elapsed_time = (time.time() - initial_time) * unit.seconds

In [16]:
elapsed_time

Quantity(value=8.10090708732605, unit=second)

# Check computation time for setLongRangeCorrection(True) for both CustomNonbondedForces

Use long-range env

In [14]:
with open('rbd_ace2_example/rbd_ace2_hybrid_system.xml', 'r') as f:
    system = XmlSerializer.deserialize(f.read())
with open('rbd_ace2_example/rbd_ace2_state.xml', 'r') as f:
    state = XmlSerializer.deserialize(f.read())

In [16]:
system.getForce(7).getUseDispersionCorrection()

True

In [15]:
system.getForce(8).setUseLongRangeCorrection(True)

In [17]:
# Set up integrator
integrator = openmmtools.integrators.LangevinIntegrator(temperature=300 * unit.kelvin, 
                                                        collision_rate=1 / unit.picoseconds, 
                                                        timestep=4 * unit.femtoseconds)


In [18]:
# Set up context
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*state.getPeriodicBoxVectors())
context.setPositions(state.getPositions())
context.setVelocitiesToTemperature(300 * unit.kelvin)

In [19]:
nsteps = 100
initial_time = time.time()
for step in range(nsteps):
    context.setParameter('lambda_sterics_insert', (step + 1) * 0.01)
    integrator.step(1)
elapsed_time = (time.time() - initial_time) * unit.seconds

In [20]:
elapsed_time

Quantity(value=36.05855202674866, unit=second)

# Check computation time for setLongRangeCorrection(True) for both CustomNonbondedForces (and with Peter's fix)

Use long-range-peter env

In [2]:
with open('rbd_ace2_example/rbd_ace2_hybrid_system.xml', 'r') as f:
    system = XmlSerializer.deserialize(f.read())
with open('rbd_ace2_example/rbd_ace2_state.xml', 'r') as f:
    state = XmlSerializer.deserialize(f.read())

In [3]:
system.getForce(7).getUseDispersionCorrection()

True

In [4]:
system.getForce(8).setUseLongRangeCorrection(True)

In [5]:
# Set up integrator
integrator = openmmtools.integrators.LangevinIntegrator(temperature=300 * unit.kelvin, 
                                                        collision_rate=1 / unit.picoseconds, 
                                                        timestep=4 * unit.femtoseconds)


In [6]:
# Set up context
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*state.getPeriodicBoxVectors())
context.setPositions(state.getPositions())
context.setVelocitiesToTemperature(300 * unit.kelvin)

In [7]:
nsteps = 100
initial_time = time.time()
for step in range(nsteps):
    context.setParameter('lambda_sterics_insert', (step + 1) * 0.01)
    integrator.step(1)
elapsed_time = (time.time() - initial_time) * unit.seconds

In [8]:
elapsed_time

Quantity(value=12.859097957611084, unit=second)